In [2]:
from PIL import Image
from scipy.io import loadmat
import numpy as np
from pathlib import Path
from tqdm import tqdm
import itertools
from scipy.ndimage.morphology import distance_transform_edt
from collections import Counter
from itertools import chain
import json

                        
def boundary_length_distribution(folder):
    lengths = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                lengths.append(x_inds[-1] - x_inds[0])
    return lengths


def boundary_middle_distribution(folder):
    middles = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                middles.append(x_inds[len(x_inds) // 2])
    return middles

# Visualizers

In [ ]:
def show_layers_from_boundary_array(img_array, layer_array, fluid=None):
    err_msg = "layer boundaries not compatible with image width"
    assert img_array.shape[1] == layer_array.shape[0], err_msg
    dme_colorcode = {
        1: (170, 160, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100),
        8: (180, 255, 255)  # fluid
    }
    amd_colorcode = {
        1: (180, 200, 250),
        2: (120, 200, 250),
    }
    zeros = np.zeros_like(img_array, dtype="uint8")
    hue = np.zeros_like(img_array, dtype="uint8")
    saturation = np.zeros_like(img_array, dtype="uint8")
    value = np.zeros_like(img_array, dtype="uint8")
    mask = np.zeros(img_array.shape, dtype="uint8")
    for w in range(img_array.shape[1]):
        if ~np.isnan(layer_array[w, :]).any():
            last_boundary = int(layer_array[w, 0])
            for idx, h in enumerate(layer_array[w, 1:]):
                curr_boundary = int(h) + 1
                mask[last_boundary:curr_boundary, w] = idx + 1
                last_boundary = curr_boundary
    if fluid is not None:
        mask[fluid != 0] = 8
    if layer_array.shape[1] == 8:
        for klass, hsv in dme_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    if layer_array.shape[1] == 3:
        for klass, hsv in amd_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    alpha = np.zeros_like(img_array, dtype="uint8")
    alpha[mask != 0] = 255
    img_stack = np.array([zeros, zeros, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    alphaimg = Image.fromarray(alpha)
    img_with_boundaries = Image.composite(colored_mask, img, alphaimg)
    return img_with_boundaries


def show_boundary_from_boundary_array(img_array, layer_array):
    err_msg = "layer boundaries not compatible with image width"
    assert img_array.shape[1] == layer_array.shape[0], err_msg
    mask = np.zeros(img_array.shape, dtype="uint8")
    for w_idx in range(img_array.shape[1]):
        if ~np.isnan(layer_array[w_idx, :]).any():
            for h_idx in layer_array[w_idx, :]:
                mask[int(h_idx), w_idx] = 255
    empty = mask.copy()
    ones = np.ones(img_array.shape, dtype="uint8") * 255
    img_stack = np.array([empty, empty, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([empty, ones, ones]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    maskimg = Image.fromarray(mask)
    img_with_boundaries = Image.composite(colored_mask, img, maskimg)
    return img_with_boundaries


def show_layers_from_mask_array(img, mask):
    err_msg = "image and mask do not have the same dimensions"
    assert img.shape == mask.shape, err_msg
    dme_colorcode = {
        1: (170, 160, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100),
        9: (180, 255, 255)  # fluid
    }
    zeros = np.zeros_like(mask, dtype="uint8")
    hue = zeros.copy()
    saturation = zeros.copy()
    value = zeros.copy()
    alpha = zeros.copy()
    for klass, hsv in dme_colorcode.items():
        hue[mask == klass] = hsv[0]
        saturation[mask == klass] = hsv[1]
        value[mask == klass] = hsv[2]
        alpha[mask == klass] = 255
    img_stack = np.array([zeros, zeros, img]).transpose((1, 2, 0))
    img_img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    mask_img = Image.fromarray(colored_mask_stack, mode="HSV")
    alpha_img = Image.fromarray(alpha)
    img_w_layers = Image.composite(mask_img, img_img, alpha_img)
    return img_w_layers

# Data generation

In [ ]:
def create_patches(img, lyr, patch_width):
    idx = np.where((np.isnan(lyr)).any(axis=0))[0]
    diff = np.diff(idx)
    useful_parts = diff >= patch_width
    useful_lengths = diff[useful_parts]
    useful_start_idx = idx[np.pad(useful_parts, (0, 1), constant_values=[False])]
    for ustart, ulength in zip(useful_start_idx, useful_lengths):
        number_of_shifts = (ulength - 1) // patch_width
        for shift_idx in range(number_of_shifts):
            img_patch = img[:, ustart + 1 + shift_idx * patch_width:ustart + 1 + (shift_idx + 1) * patch_width]
            lyr_patch = lyr[:, ustart + 1 + shift_idx * patch_width:ustart + 1 + (shift_idx + 1) * patch_width]
            mask = create_layer_mask(lyr_patch, img_patch.shape[0])
            yield img_patch, mask
            
def generate_dme_dataset(input_dir, output_dir, patch_width):
    files = list(Path(input_dir).glob("*"))
    output_dir = Path(output_dir)
    output_dir.mkdir()
    cnt = 0
    for file in tqdm(files, desc="data generation"):
        data = loadmat(file)
        layers = data["manualLayers1"].transpose((2, 0, 1))
        images = data["images"].transpose((2, 0, 1))
        for idx, (image, layer) in enumerate(zip(images, layers)):
            patch_generator = create_patches(image, layer, patch_width)
            for img, mask in patch_generator:
                Image.fromarray(img).save(output_dir / f"img_{cnt}.png")
                Image.fromarray(mask).save(output_dir / f"mask_{cnt}.png")
                cnt += 1
                
def create_boundary_mask(boundary_array, height):
    mask = np.zeros((height, boundary_array.shape[1]), dtype="uint8")
    for col_idx, col in enumerate(boundary_array.T):
        if ~np.isnan(col).any():
            for boundary in col:
                mask[int(boundary), col_idx] = 1
    return mask

def create_layer_mask(boundary_array, height):
    mask = np.zeros((height, boundary_array.shape[1]), dtype="uint8")
    for col_idx, col in enumerate(boundary_array.T):
        prev_boundary = 0
        for boundary_idx, boundary in enumerate(col):
            mask[prev_boundary:int(boundary) + 1, col_idx] = boundary_idx
            prev_boundary = int(boundary) + 1
        mask[prev_boundary:, col_idx] = boundary_idx + 1
    return mask

def create_patches(img, lyr, patch_width):
    idx = np.where((np.isnan(lyr)).any(axis=0))[0]
    diff = np.diff(idx)
    useful_parts = diff >= patch_width
    useful_lengths = diff[useful_parts]
    useful_start_idx = idx[np.pad(useful_parts, (0, 1), constant_values=[False])]
    for ustart, ulength in zip(useful_start_idx, useful_lengths):
        number_of_shifts = (ulength - 1) // patch_width
        for shift_idx in range(number_of_shifts):
            img_patch = img[:, ustart + 1 + shift_idx * patch_width:ustart + 1 + (shift_idx + 1) * patch_width]
            lyr_patch = lyr[:, ustart + 1 + shift_idx * patch_width:ustart + 1 + (shift_idx + 1) * patch_width]
#             lyr_patch_padded = np.pad(lyr, ((0, 1), (0, 0)), constant_values=(495,))
#             lyr_patch_padded = np.pad(lyr_patch_padded, ((1, 0), (0, 0)), constant_values=(0,))
#             lyr_patch_widths = np.diff(lyr_patch_padded)
            mask = create_layer_mask(lyr_patch, img_patch.shape[0])
            yield img_patch, mask, lyr_patch.T.tolist()#, lyr_patch_widths.T.tolist()
            
def generate_dme_dataset(input_dir, output_dir, patch_width):
    files = list(Path(input_dir).glob("*"))
    output_dir = Path(output_dir)
    output_dir.mkdir()
    cnt = 0
    boundary_indices_dict = {}
    layer_widths_dict = {}
    for file in tqdm(files, desc="data generation"):
        data = loadmat(file)
        layers = data["manualLayers1"].transpose((2, 0, 1))
        images = data["images"].transpose((2, 0, 1))
        for idx, (image, layer) in enumerate(zip(images, layers)):
            patch_generator = create_patches(image, layer, patch_width)
            for img, mask, boundary_indices_list, layer_widths_list in patch_generator:
                Image.fromarray(img).save(output_dir / f"img_{cnt}.png")
                Image.fromarray(mask).save(output_dir / f"mask_{cnt}.png")
                boundary_indices_dict[cnt] = boundary_indices_list
                layer_widths_dict[cnt] = layer_widths_list
                cnt += 1
    with open(output_dir / "boundary_indices.json", "w") as boundary_file:
        json.dump(boundary_indices_dict, boundary_file)
#     with open(output_dir / "layer_widths.json", "w") as layer_widths_file:
#         json.dump(layer_widths_dict, layer_widths_file)

# LOAD FILES

In [2]:
amd = list(Path("../../dataset/raw/AMD/").glob("*"))
control = list(Path("../../dataset/raw/Control/").glob("*"))
dme = list(Path("../../dataset/raw/2015_BOE_Chiu/").glob("*"))

# Test new concept

# DME

In [12]:
data = loadmat(dme[2])
idx = 35
img = data["images"][:, :, idx]
lyr = data["manualLayers1"][:, :, idx]
lyr2 = data["manualLayers2"][:, :, idx]

In [41]:
def show_layers_from_mask_array(img, mask):
    err_msg = "image and mask do not have the same dimensions"
    assert img.shape == mask.shape, err_msg
    dme_colorcode = {
        1: (170, 160, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100),
        9: (180, 255, 255)  # fluid
    }
    zeros = np.zeros_like(mask, dtype="uint8")
    hue = zeros.copy()
    saturation = zeros.copy()
    value = zeros.copy()
    alpha = zeros.copy()
    for klass, hsv in dme_colorcode.items():
        hue[mask == klass] = hsv[0]
        saturation[mask == klass] = hsv[1]
        value[mask == klass] = hsv[2]
        alpha[mask == klass] = 200
    img_stack = np.array([zeros, zeros, img]).transpose((1, 2, 0))
    img_img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    mask_img = Image.fromarray(colored_mask_stack, mode="HSV")
    alpha_img = Image.fromarray(alpha)
    img_w_layers = Image.composite(mask_img, img_img, alpha_img)
    return img_w_layers

In [44]:
idx = 12
img = np.array(Image.open(f"../../generated/DME_496/img_{idx}.png"))
lyr = np.array(Image.open(f"../../generated/DME_496/mask_{idx}.png"))

In [46]:
show_layers_from_mask_array(img, lyr).show()

In [13]:
show_layers_from_boundary_array(img, lyr2.T).show()
show_layers_from_boundary_array(img, lyr.T).show()

In [320]:
data = loadmat("../../dataset/raw/2015_BOE_Chiu/Subject_02.mat")

In [321]:
idx = 28

In [322]:
img = data["images"][..., idx]
lyr = data["manualLayers1"][..., idx]

In [323]:
show_layers_from_boundary_array(img, lyr.T).show()

In [65]:
segmented_length = list()

for d in range(10):
    data = loadmat(dme[d])
    for lyr_ind in range(61):
        m1 = data["manualLayers1"][..., lyr_ind]
        m2 = data["manualLayers2"][..., lyr_ind]
        lengths = [] 
        if (~np.isnan(m1)).any():
            w_idx = np.where(~np.isnan(m1))[1]
            delta = w_idx[-1] - w_idx[0]
            lengths.append(delta)
        else:
            lengths.append(0)
        if (~np.isnan(m2)).any():
            w_idx = np.where(~np.isnan(m2))[1]
            delta = w_idx[-1] - w_idx[0]
            lengths.append(delta)
        else:
            lengths.append(0)
        if sum(lengths):
            segmented_length.append(tuple(lengths))
            
Counter(segmented_length)

Counter({(523, 523): 33,
         (547, 547): 11,
         (535, 535): 11,
         (541, 541): 33,
         (505, 505): 11,
         (499, 499): 11})

In [5]:
dme496img = sorted(list(Path("../../generated/DME_496/").glob("img*")))
dme496mask = sorted(list(Path("../../generated/DME_496").glob("mask*")))

In [6]:
img = np.array(Image.open(dme496img[0]))
mask = np.array(Image.open(dme496mask[0]))

In [8]:
show_layers_from_mask_array(img, mask).show()

# There are parts where not all the layers are annotated

In [6]:
# for d in range(10):
#     data = loadmat(dme[d])
#     lyr = data["manualLayers1"]
#     if (np.isnan(lyr).any(axis=0) != np.isnan(lyr).all(axis=0)).sum():
#         print(d)

data = loadmat(dme[0])
lyr = data["manualLayers1"]
img = data["images"]
idx = (np.isnan(lyr).any(axis=0) != np.isnan(lyr).all(axis=0))
np.isnan(lyr[:, idx]).sum(axis=-1)

array([  0,   0,   0,   0, 127,   0,   0,   0])

# Generating the dataset

In [6]:
patch_width = 70
generate_dme_dataset("../../dataset/raw/2015_BOE_Chiu", f"../../generated/DME_{patch_width}", patch_width)

data generation: 100%|██████████| 10/10 [00:19<00:00,  1.95s/it]


In [4]:
len(list(Path(f"../../generated/DME_{patch_width}").glob("img*")))

954

In [7]:
img.shape

(496, 768, 61)

# Proove that the layer mask generation is correct

In [354]:
bmask = create_boundary_mask(lyr[:, 500:600], 496)
lmask = create_layer_mask(lyr[:, 500:600], 496)
diff = np.diff(lmask, axis=0)
(np.pad(diff, ((0, 1), (0, 0)), constant_values=(0,)) != bmask).sum()

# AMD

In [10]:
data = loadmat(amd[0])
idx = 30
img = data["images"][:, :, idx]
lyr = data["layerMaps"][idx]

In [14]:
def generate_amd_dataset(amd_dir, control_dir, output_dir, patch_width):
    files = list(
        chain(
            Path(amd_dir).glob("*"),
            Path(control_dir).glob("*")
        )
    )
    output_dir = Path(output_dir)
    output_dir.mkdir()
    cnt = 0
    for file in tqdm(files, desc="data generation"):
        data = loadmat(file)
        layers = data["layerMaps"].transpose((0, 2, 1))
        images = data["images"].transpose((2, 0, 1))
        for idx, (image, layer) in enumerate(zip(images, layers)):
            patch_generator = create_patches(image, layer, patch_width)
            for img, mask in patch_generator:
                Image.fromarray(img).save(output_dir / f"img_{cnt}.png")
                Image.fromarray(mask).save(output_dir / f"mask_{cnt}.png")
                cnt += 1

In [34]:
patch_width = 736
generate_amd_dataset(
    "../../dataset/raw/AMD/",
    "../../dataset/raw/Control/",
    f"../../generated/AMD_{patch_width}",
    patch_width
)

data generation: 100%|██████████| 384/384 [04:31<00:00,  1.42it/s]


In [35]:
len(list(Path("../../generated/AMD_736").glob("img*")))

2901

In [26]:
loadmat(d)

784

In [33]:
736 / 16

46.0

In [11]:
show_layers_from_boundary_array(img, lyr).show()
show_boundary_from_boundary_array(img, lyr).show()